In [105]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

from imblearn.over_sampling import SMOTE
#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [106]:
def get_clf_eval(y_test, y_data, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_val(y_data)
    recall = recall_val(y_data) 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
#     confusion = confusion_matrix(y_test, pred) 
#     accuracy = accuracy_score(y_test, pred) 
#     precision = precision_score(y_test, pred , average= 'macro') 
#     recall = recall_score(y_test, pred , average= 'macro') 
#     f1 = f1_score(y_test, pred , average= 'macro') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
    return [accuracy, precision, recall, f1]

In [107]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

In [108]:
def recall_val(y_data):

    real = y_data[y_data['실제 y라벨링'] > 2]
    new = real[real['예측 y 라벨링'] > 2]
    if len(real) == 0:
        per = 0
    else :
        per = len(new)/len(real)
    return per

In [109]:
def precision_val(y_data):
    

    new = y_data[y_data['예측 y 라벨링'] > 2]
    real = new[new['실제 y라벨링'] > 2]
    if len(new) == 0:
        per = 0
    else :
        per = len(real)/len(new)
    return per

In [110]:
def mean(y_data):
    portfolio = []
    for i in range(len(y_data)):
        buy = y_data[i]['실제 y 수익률']
        sum_1 = buy.sum()
        sum_2 = len(buy)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "all수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
    return data

In [111]:
def buy(y_data):
    portfolio = []
    for i in range(len(y_data)):
        buy = y_data[i][y_data[i]['예측 y 라벨링'] > 2]['실제 y 수익률']
        sum_1 = buy.sum()
        sum_2 = len(buy)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "buy수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
                         
    return data

In [112]:
def buy_sell(y_data):
    portfolio = []
    for i in range(len(y_data)):
        buy = y_data[i][y_data[i]['예측 y 라벨링'] > 2]['실제 y 수익률']
        sell = y_data[i][y_data[i]['예측 y 라벨링'] < 2]['실제 y 수익률']
        sum_1 = buy.sum() - sell.sum()
        sum_2 = len(buy)+len(sell)
        value = sum_1/sum_2
        portfolio.append(value)
    data = pd.DataFrame(portfolio).T
    data.rename(index = {0 : "buy_sell수익률"},columns = lambda x : "model_set_"+ str(x),inplace = True )
    data['mean'] = data.mean(axis=1)[0]
                         
    return data

In [113]:
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv",encoding = "cp949")

In [114]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률','IPO_PER','IPO_PBR','IPO_EV/EBITDA'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률','IPO_PER','IPO_PBR','IPO_EV/EBITDA'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률','IPO_PER','IPO_PBR','IPO_EV/EBITDA'],axis =1)

In [115]:
def process_final(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 4 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [116]:
df_score_0_1,df_feature_0_1,y_data_0_1 = process_final(df_0_1,'공모가 대비 1개월 수익률')
df_score_0_3,df_feature_0_3,y_data_0_3 = process_final(df_0_3,'공모가 대비 3개월 수익률')
df_score_0_6,df_feature_0_6,y_data_0_6 = process_final(df_0_6,'공모가 대비 6개월 수익률')

In [117]:
df_score_0_1.to_csv("D:/공모전/IPO/IPO/Result/랜덤포레스트/모델 평가/new_0_1_모델 평가.csv",encoding = 'euc-kr')
df_score_0_3.to_csv("D:/공모전/IPO/IPO/Result/랜덤포레스트/모델 평가/new_0_3_모델 평가.csv",encoding = 'euc-kr')
df_score_0_6.to_csv("D:/공모전/IPO/IPO/Result/랜덤포레스트/모델 평가/new_0_6_모델 평가.csv",encoding = 'euc-kr')

In [118]:
import pandas as pd
import numpy as np
import os

In [119]:
from scipy import stats
def Ttest(x,y, Number = 0 , leftstr = "", rightstr= "", Measure = ""):
    print(str(Number)+"\t"+leftstr + " & " + rightstr + "\tMeasure : " + Measure + "\n")
    _stat,_pvalue = stats.levene(x,y)
    print("LeveneResult -- stat : %3f, p-value : %3f \n" %(_stat, _pvalue))
    if _pvalue < 0.05:
        equal_Var = False
    else :
        equal_Var = True
    print(equal_Var)
    statistic , pvalue = stats.ttest_ind(x,y, equal_var= equal_Var)
    
    print("\nstatistic : %d , pvalue : %.7f\n" % (statistic, pvalue))
    return (statistic, pvalue)

In [120]:
def sign(pval):
    if(pval <= 0.01):
        return "***"
    elif(pval <= 0.05):
        return "**"
    elif(pval<= 0.1):
        return "*"
    else:
        return ""

In [121]:
path_dir = 'Result/랜덤포레스트/모델 평가' 

In [122]:
file_list = [ 'new_0_3_모델 평가.csv', 'new_0_6_모델 평가.csv']

In [123]:
df_0_1 = pd.read_csv("Result/랜덤포레스트/모델 평가/new_0_1_모델 평가.csv",index_col = 0,encoding = 'cp949')

In [124]:
df_0_1['mean'] = df_0_1.mean(axis =1)
first = df_0_1['mean']
first

정확도              0.598408
정밀도              0.631915
재현율              0.723997
f1_score         0.609116
정확도_test         0.325971
정밀도_test         0.426354
재현율_test         0.546971
f1_score_test    0.291458
Name: mean, dtype: float64

In [125]:

for file in file_list:
    df = pd.read_csv(path_dir + "//" + file, encoding = 'cp949',index_col = 0)
    df["mean"] = df.mean(axis=1)
    mean = df['mean']
    

    first = pd.concat([first,mean],axis = 1)

In [127]:
first.columns = ["공모가 대비 1개월","공모가 대비 3개월","공모가 대비 6개월"]

In [128]:
first.to_csv("PER")

,공모가 대비 1개월,공모가 대비 3개월,공모가 대비 6개월
정확도,0.598408,0.612240,0.604466
정밀도,0.631915,0.679439,0.689258
재현율,0.723997,0.740106,0.738439
f1_score,0.609116,0.618414,0.609701
정확도_test,0.325971,0.293477,0.280512
정밀도_test,0.426354,0.459996,0.469492
재현율_test,0.546971,0.527352,0.563756
f1_score_test,0.291458,0.246874,0.253727
